In [0]:
%sql
create schema if not exists bike_data.silver

In [0]:
df=spark.table('bike_data.bronze.cust_info_crm')

In [0]:
df.display()

### Checking Dupliactes

In [0]:
from pyspark.sql.functions import *

In [0]:
df.groupBy("cst_id") \
  .count() \
  .filter("count > 1") \
  .display()

In [0]:
df=df.dropDuplicates(["cst_id"])

In [0]:
df.groupBy("cst_id") \
  .count() \
  .filter("count > 1") \
  .display()

In [0]:
df.describe()

In [0]:
df.display()

#### triming columns

In [0]:
df = df.withColumn("cst_firstname",trim(col("cst_firstname")))\
    .withColumn("cst_lastname",trim(col("cst_firstname")))



In [0]:
df.display()

In [0]:
df.count()

### Handling Null Values 

In [0]:
df.select(
    *[
        sum(col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ]
).display()

In [0]:
df = df.dropna(subset=["cst_id","cst_firstname"])

In [0]:
df= df.dropna("any")

In [0]:
df.count()

In [0]:
df.describe()

#### Changing datatype

In [0]:
df=df.withColumn("cst_id",col("cst_id").cast("int"))


In [0]:
df.display()

In [0]:
df.schema

#### Changing Column names

In [0]:
for c in df.columns:
    if c.startswith("cst"):
        df = df.withColumnRenamed(c, c.replace("cst", "customer", 1))


In [0]:
df.display()

In [0]:
# 10. Write Silver
df.write \
    .format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("bike_data.silver.dim_customer_crm")